In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge

In [3]:
from sklearn.metrics import mean_squared_error

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import geopandas as gpd
import shapely
from sodapy import Socrata
import pysal.lib

C:\Users\rob\anaconda3\envs\spatial\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\rob\AppData\Local\Temp\ipykernel_18060\378036255.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedoc

In [7]:
neighbor = gpd.GeoDataFrame.from_file('../geo/neighborhoods.geojson')
comm = pd.read_csv('../geo/zip_comm.csv')
census = pd.read_csv('../geo/census_zip.csv')

In [8]:
comm['ZCTA5'] = comm['ZCTA5'].astype('int').astype('str')
comm['CHGOCA'] = comm['CHGOCA'].astype('int').astype('str')

In [9]:
census['zip'] = census['zip'].astype('int').astype('str')
census['tract'] = census['tract'].astype('str')

In [10]:
data = ['EMSOD','HCSBDP','HCSBHP','HCSNVP',
             'HCSSPDP','HDBHR','HDDRR','HDSZR',
        'MEODR','YRDEP','VRALR','VRDIDR', 'VRDOR', 'VRSUR']

In [11]:
files = [f'../data/{file}.csv' for file in data]

In [12]:
mh = pd.concat(map(pd.read_csv, files),axis = 0)
mh = mh[mh['p'].isnull()]
mh = mh[mh['l'] != 'place']
mh['g'] = mh['g'].astype('str')

mh = mh.merge(comm, how = 'left', left_on = 'g', right_on = 'ZCTA5')

In [13]:
mh.loc[mh['g'].str.len() == 5,'g'] = mh['CHGOCA']
mh['g'] = mh['g'].str.replace('1714000-','')

In [14]:
neighbor = neighbor.merge(mh, how = 'left', left_on = 'area_numbe', right_on = 'g')

In [83]:
neighbor['long'] = neighbor['geometry'].centroid.x
neighbor['lat'] = neighbor['geometry'].centroid.y

C:\Users\rob\AppData\Local\Temp\ipykernel_18060\498829464.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighbor['long'] = neighbor['geometry'].centroid.x
C:\Users\rob\AppData\Local\Temp\ipykernel_18060\498829464.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighbor['lat'] = neighbor['geometry'].centroid.y


----

##  CZA

In [150]:
CZA = pd.read_csv(f'../data/CZA.csv')
CZB = pd.read_csv(f'../data/CZB.csv')
CZD = pd.read_csv(f'../data/CZD.csv')
CZH = pd.read_csv(f'../data/CZH.csv')
CZL = pd.read_csv(f'../data/CZL.csv')
CZM = pd.read_csv(f'../data/CZM.csv')
CZO = pd.read_csv(f'../data/CZO.csv')
CZP = pd.read_csv(f'../data/CZP.csv')
CZR = pd.read_csv(f'../data/CZR.csv')
CZS = pd.read_csv(f'../data/CZS.csv')
CZT = pd.read_csv(f'../data/CZT.csv')
CZU = pd.read_csv(f'../data/CZU.csv')
CZV = pd.read_csv(f'../data/CZV.csv')

In [146]:
df = CZA

In [171]:
def transform(df):
    df.drop(df[df['p'].notnull()].index, inplace = True)
    df.drop(df[df['l'] == 'place'].index, inplace = True)

    df = df.merge(census, how = 'left', left_on = 'g', right_on = 'tract')
    df['zip'] = df['zip'].astype('str')
    df.loc[df['l'] == 'tract-2010','g'] = df['zip']

    df = df.merge(comm, how = 'left', left_on = 'g', right_on = 'ZCTA5')
    df['zip'] = df['zip'].astype('str')
    df.loc[df['l'] == 'tract-2010','g'] = df['zip']

    df.loc[df['g'].str.len() == 5,'g'] = df['CHGOCA']
    df['g'] = df['g'].str.replace('1714000-','')

    df = neighbor.merge(df, how = 'left', left_on = ['g','d'], right_on = ['g','d'])
    df.dropna(subset = 'v_y',inplace = True)
    df.rename(columns = {'a_x':'mh_stat','v_x':'mh_val','v_y':'crim_val'}, inplace = True)
    return df

In [186]:
values = []
for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    poly = PolynomialFeatures()
    X_train = poly.fit_transform(X_train)
    X_test = poly.transform(X_test)
    
    base = [np.mean(y_train)] * len(y_test)
    baseline = mean_squared_error(y_test, base, squared=False)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr_train = mean_squared_error(y_train, lr.predict(X_train), squared=False)
    lr_test = mean_squared_error(y_test, lr.predict(X_test), squared=False)
    print(('lr',lr_train,lr_test))
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    lasso_train = mean_squared_error(y_train,lasso.predict(X_train), squared = False)
    lasso_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)
    print(('lasso',lasso_train, lasso_test))
    
    rg = Ridge()
    rg.fit(X_train, y_train)
    rg_train = mean_squared_error(y_train,rg.predict(X_train), squared = False) 
    rg_test = mean_squared_error(y_test, rg.predict(X_test), squared=False)
    print(('rg',rg_train,rg_test))
    
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    knn_train = mean_squared_error(y_train,knn.predict(X_train), squared = False)
    knn_test = mean_squared_error(y_test, knn.predict(X_test), squared=False)
    print(('knn',knn_train,knn_test))

    dt = DecisionTreeRegressor()
    dt.fit(X_train, y_train)
    dt_train = mean_squared_error(y_train,dt.predict(X_train), squared = False) 
    dt_test = mean_squared_error(y_test, dt.predict(X_test), squared=False)
    print(('dt',dt_train,dt_test))

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    rf_train = mean_squared_error(y_train,rf.predict(X_train), squared = False)
    rf_test = mean_squared_error(y_test, rf.predict(X_test), squared=False)
    print(('rf',rf_train,rf_test))

    abr = AdaBoostRegressor()
    abr.fit(X_train, y_train)
    abr_train = mean_squared_error(y_train,abr.predict(X_train), squared = False)
    abr_test = mean_squared_error(y_test, abr.predict(X_test), squared=False)
    print(('abr',abr_train,abr_test))
    values.append([baseline, lr_train, lr_test, lasso_train, lasso_test, rg_train, rg_test, knn_train, knn_test, dt_train, dt_test, rf_train, rf_test, abr_train, abr_test])

('lr', 10.534465983556602, 10.449511081283774)
('lasso', 11.805790967379133, 11.764964385227657)
('rg', 10.532918472290946, 10.451252699398585)
('knn', 1.530250627707911e-15, 1.5060851752206681e-15)
('dt', 8.818193050462454e-14, 8.493403924125657e-14)
('rf', 6.062233380680906e-14, 6.038097041348159e-14)
('abr', 5.82992504817946, 5.831386582890593)
('lr', 6.052938431525064, 6.114156956941509)
('lasso', 6.415198786749262, 6.4588279866936364)
('rg', 6.04812560828611, 6.110624670128594)
('knn', 7.566608288108627e-16, 7.525981095821164e-16)
('dt', 4.741963615875575e-14, 4.6121668292788463e-14)
('rf', 0.0003606083304017734, 0.0005099771915587499)
('abr', 3.3401490937432534, 3.2961731914711603)
('lr', 20.87022046687384, 20.904542488844516)
('lasso', 21.83696941528489, 21.956253725841975)
('rg', 20.716785905083267, 20.767291004792067)
('knn', 2.1254581338350098e-15, 2.1878692845711333e-15)
('dt', 6.696770191180607e-14, 6.599484758328037e-14)
('rf', 7.726805505315927e-06, 1.21055777135188e-05)


In [195]:
errors = pd.DataFrame(values, columns = ['baseline', 'lr_train', 'lr_test', 'lasso_train', 'lasso_test', 
                                'rg_train', 'rg_test', 'knn_train', 'knn_test', 
                                'dt_train', 'dt_test', 'rf_train', 'rf_test', 
                                'abr_train', 'abr_test'])

In [200]:
columns = pd.DataFrame(['CZA','CZB', 'CZD', 'CZH', 'CZL','CZM', 'CZO', 'CZP', 'CZR', 'CZS', 'CZT', 'CZV'])

In [202]:
errors = pd.concat([errors,columns], axis = 1)

In [213]:
errors.set_index(0, inplace = True)

In [215]:
errors.to_csv('../data/errors.csv')

In [218]:
errors.describe()

,baseline,lr_train,lr_test,lasso_train,lasso_test,rg_train,rg_test,knn_train,knn_test,dt_train,dt_test,rf_train,rf_test,abr_train,abr_test
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,12.000000,12.000000
mean,28.990317,24.643178,24.194683,25.305752,24.841951,24.588915,24.151797,3.739697e-15,3.691298e-15,1.555079e-13,1.561268e-13,6.451507e-05,1.006740e-04,8.539349,8.479723
std,32.644718,31.236994,30.525501,31.247112,30.523201,31.189247,30.493490,5.039662e-15,5.017766e-15,2.024397e-13,2.050298e-13,1.209356e-04,1.840148e-04,9.109619,9.040950
min,0.528688,0.284719,0.283765,0.527336,0.528688,0.282713,0.281901,5.099318e-17,5.138775e-17,1.808525e-15,1.792205e-15,6.154567e-15,6.106737e-15,0.189864,0.191351
25%,6.196792,4.317524,4.343703,4.878107,4.911553,4.270108,4.303217,6.035786e-16,5.984662e-16,3.792883e-14,3.686074e-14,1.789910e-13,1.804990e-13,2.048689,2.056417
50%,14.331292,8.773222,8.715152,9.736370,9.702697,8.751475,8.699843,1.243998e-15,1.229857e-15,5.960357e-14,5.839447e-14,1.943396e-06,2.470013e-06,4.585037,4.563780
75%,40.757704,32.891653,32.296971,33.767740,33.201759,32.775844,32.196010,4.848978e-15,4.843154e-15,1.977226e-13,2.003337e-13,4.826140e-05,7.902824e-05,13.405327,13.148415
max,88.575164,80.512624,80.656812,80.981111,81.546501,80.218793,80.677648,1.496712e-14,1.574919e-14,6.099336e-13,6.198657e-13,3.606083e-04,5.099772e-04,25.055669,24.786710


In [ ]:
for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    poly = PolynomialFeatures()
    X_train = poly.fit_transform(X_train)
    X_test = poly.transform(X_test)
    
    gs = GridSearchCV(KNeighborsRegressor(),
                      param_grid = {'n_neighbors':[3,5,7,10,15],
                                   'p':[1,2]},
                      cv = 5,
                      n_jobs = -1)
    
    gs.fit(X_train, y_train)
    print(gs.best_params_)

{'n_neighbors': 3, 'p': 1}


In [ ]:
, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV

In [221]:
for stat in [CZA]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    poly = PolynomialFeatures()
    X_train = poly.fit_transform(X_train)
    X_test = poly.transform(X_test)
    
    knn = KNeighborsRegressor(n_neighbors=3)
    knn.fit(X_train,y_train)
    y_pred = knn.predict(X_test)

In [223]:
[X_test,y_pred]

[array([[ 1.        , -0.72260815, -1.51503973, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.091508  , -0.48239101, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.4717863 ,  1.03157575, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        ...,
        [ 1.        ,  2.837444  ,  0.4378657 , ...,  3.01478808,
         -1.0050396 ,  0.33504995],
        [ 1.        , -0.3544664 ,  1.23083567, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.86824665, -0.27181914, ...,  0.33169827,
         -0.99498567,  2.98462961]]),
 array([50.63247933, 18.938757  ,  7.79390601, ..., 63.73305938,
        13.3781609 , 23.45584757])]